In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import pandas as pd

In [7]:
data_df = pd.read_csv('../dataset/anno/img_label.csv',sep=',')
data_df['category_label'] = data_df['category_label'].astype(str)
data_df.head()

,image_name,category_label
0,../dataset/img/High-Neck_Blouse/img_00000041.jpg,1
1,../dataset/img/Strappy-Back_Yoga_Romper/img_00...,16
2,../dataset/img/Classic_Capri_Leggings/img_0000...,11
3,../dataset/img/Caged-Back_Longline_Tank/img_00...,6
4,../dataset/img/Floral_Lace_Maxi_Cardigan/img_0...,2


In [12]:
input_size = (224,224)
num_class = 17

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255, rotation_range=20, zoom_range=0.2)

train_data_gen = datagen.flow_from_dataframe(
    data_df, x_col='image_name',y_col = 'category_label',
    subset='training',target_size=input_size,directory='.',
    class_mode='categorical',batch_size=32, seed=42
)

val_data_gen = datagen.flow_from_dataframe(
    data_df, x_col='image_name',y_col = 'category_label',
    subset='validation',target_size=input_size,directory='.',
    class_mode='categorical',batch_size=32, seed=42
)

Found 69089 validated image filenames belonging to 17 classes.
Found 17272 validated image filenames belonging to 17 classes.


In [11]:
encoder = tf.keras.applications.resnet.ResNet50(weights='imagenet', include_top=True, input_shape=input_size+(3,))
encoder.output_shape

(None, 1000)

In [18]:
inp_classifier = Sequential([
    layers.Input(shape = encoder.output_shape[1:]),
    layers.Flatten(),

    layers.Dense(512, activation='relu', kernel_regularizer = regularizers.l2(1e-5)),
    layers.Dropout(0.1),

    layers.Dense(256, activation='relu', kernel_regularizer = regularizers.l2(1e-5)),
    layers.Dropout(0.1),
    
    layers.Dense(128, activation='relu', kernel_regularizer = regularizers.l2(1e-7)),
    layers.Dropout(0.1),
    
    layers.Dense(num_class, activation='softmax')
], name='classifier' )

inp_classifier.summary()


Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1000)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               512512    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

In [19]:
model = tf.keras.Model(inputs=encoder.input, outputs=inp_classifier(encoder.output),name = 'complete_model')
model.summary()

Model: "complete_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_____________________________________________________________________________________

In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(train_data_gen, epochs=10, validation_data=val_data_gen)